# Final Project

## problem

`-` 자신이 어떤 회사를 운영하거나 회사의 데이터 관리자라고 가정해보자 (예를 들어, 온라인 쇼핑몰을 운영하거나 데이터 분석 컨설팅 업체를 설립한다고 생각해볼 수 있겠다).  

`-` 회사의 데이터를 관리 및 활용하는 관점에서 데이터베이스를 설계하고, 필요한 데이터 분석 정보를 생성하는 Project를 수행하시오.

`-` 보고서에는 다음과 같은 사항이 포함되어야 합니다.

1. 필요한 데이터 항목, 데이터로부터 얻어야 되는 정보

2. 데이터베이스 구조 (테이블 수 및 테이블들간의 관계)

3. 데이터 업로드 과정 (데이터 import 과정, SQL 문장 등)

4. 데이터베이스에 업로드된 데이터 요약 (요약 통계량, 표/그래프 활용)

5. 필요한 정보를 얻기 위한 데이터 분석 (데이터 분석 기법 활용)

6. 결론 및 프로젝트 수행 소감

`-` Project 보고서는 1개의 파일로 만들어 첨부하시오. 

`-` R, Python code는 보고서에 부록으로 넣으시오.

`-` 보고서 발표 및 제출 마감일 : 2022. 6. 14(화) 22:00

## 할 것 정리

`-` 국민건강검진 데이터를 바탕으로 국민들의 건강을 check하고 건강 향상을 위한 정책이 필요한지를 판단하자

`-` 예컨대 흡연율이 증가하고 있다면 원인 파악은 주어진 데이터로는 모르겠지만 건강을 위해 금연 정책을 할 수는 있음

`-` 만약 흡연율이 감소하고 있다면 금연 정책에 들이는 노력을 다른 곳에 사용할지 고려할 수 있다

`-` 흡연율이 그대로라면 현재 시행하고 있는 금연 정책이 실효성이 있는지 파악해야 한다(흡연율이 왜 증가하는지 파악해야 함)

- 필요한 데이터 항목과 데이터로부터 얻어야 되는 정보

`-` 국민건강검진 데이터와 인구수와 병원수 데이터가 필요하다

`-` 데이터로부터 고혈압이나 당뇨 등의 비율을 알 수 있다

`-` 최근 3개년인 2018년~2020년 까지의 데이터는 일단 DB로 만들고 필요한 컬럼만 뽑아서 사용하자

`-` 예컨대 연도별 흡연율의 변화

`-` 2018~2020년 인구수 데이터 링크: https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B040A3

- 데이터베이스 구조 (테이블 수 및 테이블들간의 관계)

`-` 결측치가 많은 데이터끼리 모아 테이블을 만든다

`-` 나머지 컬럼에 대해서는 성격상 비슷한 것끼리 묶는다(내과, 안과 등)

`-` 테이블의 관계는 외래키를 통해 설명

- 나머지 part는 그냥 하면 된다

## 패키지 import

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## NHIS 데이터 불러오기

`-` 국민건강검진 데이터는 2018~2020년의 데이터를 하나의 파일로 묶겠다

`-` 결손치 유무, 제3대구치(사랑니) 이상, 치아마모증유무 변수는 2020년 데이터에는 존재하지 않는다

`-` 또한, 2018년과 2019년 데이터에서도 컬럼만 존재하지 미시행하였기 때문에 drop 하겠다

`-` 가입자 일련번호를 1~3000000으로 set 해주고 데이터 공개일자 변수는 기준년도 변수가 있으므로 drop 하겠다

In [3]:
def remove_space(df: pd.DataFrame) -> pd.DataFrame:
    """컬럼명에 포함된 공백을 제거한 데이터프레임을 반환하는 함수"""
    df = df.rename(columns = dict(zip(df.columns, [column.replace(' ', '') for column in df.columns]))) 
    return df

In [23]:
def get_NHIS_dataframe() -> pd.DataFrame:
    """
    NHIS_2018 ~ NHIS_2020 csv 파일을 불러온 후 간단한 전처리를 수행하고 merge 하여 반환하는 함수
    
    *--- 전처리 ---*
    1. NHIS_2019 데이터에 중복행이 있어 이를 제거
    2. csv 파일의 컬럼명을 서로 동일하게 해주기 위해 각 csv 파일의 컬럼명에 있는 공백을 제거
    
    *--- Data Path ---*
    .(current directory)
    |--- Data
        |--- NHIS_OPEN_GJ_2018.csv
        |--- NHIS_OPEN_GJ_2019.csv
        |--- NHIS_OPEN_GJ_2020.csv
    """
    df = pd.concat([
                    remove_space(pd.read_csv('./Data/NHIS_OPEN_GJ_2018.csv', encoding='euc=kr')), ## 2018년

                    remove_space(pd.read_csv('./Data/NHIS_OPEN_GJ_2019.csv', encoding='euc=kr', on_bad_lines='skip').
                                    drop_duplicates(). ## 중복행 제거
                                    assign(**{'가입자 일련번호': lambda df: df['가입자 일련번호'] + 1000000})), ## 2019년도는 일련번호 1000001부터 시작
                    
                    remove_space(pd.read_csv('./Data/NHIS_OPEN_GJ_2020.csv', encoding='euc=kr').
                                    assign(**{'가입자 일련번호': lambda df: df['가입자 일련번호'] + 2000000})) ## 2020년도는 일련번호 2000001부터 시작
                   ]).reset_index().drop('index', axis=1)
    return df

In [108]:
df = get_NHIS_dataframe() ## 행이 3백만개라 시간이 조금 걸린다

- 데이터프레임의 간단한 정보

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 34 columns):
 #   Column        Dtype  
---  ------        -----  
 0   기준년도          int64  
 1   가입자일련번호       int64  
 2   성별코드          int64  
 3   연령대코드(5세단위)   int64  
 4   시도코드          int64  
 5   신장(5Cm단위)     int64  
 6   체중(5Kg단위)     int64  
 7   허리둘레          float64
 8   시력(좌)         float64
 9   시력(우)         float64
 10  청력(좌)         float64
 11  청력(우)         float64
 12  수축기혈압         float64
 13  이완기혈압         float64
 14  식전혈당(공복혈당)    float64
 15  총콜레스테롤        float64
 16  트리글리세라이드      float64
 17  HDL콜레스테롤      float64
 18  LDL콜레스테롤      float64
 19  혈색소           float64
 20  요단백           float64
 21  혈청크레아티닌       float64
 22  (혈청지오티)AST    float64
 23  (혈청지오티)ALT    float64
 24  감마지티피         float64
 25  흡연상태          float64
 26  음주여부          object 
 27  구강검진수검여부      object 
 28  치아우식증유무       float64
 29  결손치유무         object 
 30  치아마모증유무       obje

- 결측치

In [110]:
df.isnull().sum()

기준년도                  0
가입자일련번호               0
성별코드                  0
연령대코드(5세단위)           0
시도코드                  0
신장(5Cm단위)             0
체중(5Kg단위)             0
허리둘레                925
시력(좌)               705
시력(우)               706
청력(좌)               620
청력(우)               625
수축기혈압             18686
이완기혈압             18689
식전혈당(공복혈당)        19040
총콜레스테롤          1931389
트리글리세라이드        1931386
HDL콜레스테롤        1931398
LDL콜레스테롤        1952503
혈색소               19057
요단백               33017
혈청크레아티닌           19034
(혈청지오티)AST        19029
(혈청지오티)ALT        19031
감마지티피             19042
흡연상태                743
음주여부             710221
구강검진수검여부              0
치아우식증유무         1870945
결손치유무           2000000
치아마모증유무         2000000
제3대구치(사랑니)이상    2000000
치석              1893308
데이터공개일자               0
dtype: int64

- 전처리

`-` 위에서 언급했듯이 결손치 유무, 제3대구치(사랑니) 이상, 치아마모증유무, 데이터 공개일자 변수는 drop 하겠다

In [121]:
def drop_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    drop_columns 목록에 존재하는 변수를 drop한 데이터프레임을 반환하는 함수(원본이 변한다)
    """
    drop_columns = ['결손치유무', '제3대구치(사랑니)이상', '치아마모증유무', '데이터공개일자']
    for col in drop_columns:
        try:
            df = df.drop([col], axis=1)        
        except:
            pass
    return df

In [122]:
df = drop_columns(df) 

`-` 음주 여부, 구강검진 수검여부, 치석변수는 unique한 값으로 `0, 1, 'Y', 'N'` 를 가지는데 편의상 `'Y' -> 1`로 `'N' -> 0`으로 바꿔주겠다

In [123]:
def bool2int(df: pd.DataFrame) -> pd.DataFrame:
    """변수가 가지는 값에서 'Y' -> 1, 'N' -> 0으로 mapping하고 해당 데이터프레임을 반환(원본이 변한다)"""
    columns = ['음주여부', '구강검진수검여부', '치석']
    for col in columns:
        df[col] = df[col].map({'Y':1, 'N':0, 1:1, 0:0})
    return df

In [124]:
df = bool2int(df)

- 테이블 나누기(MedicalCheckup)

`-` 전처리가 끝난 csv 파일은 db로 올려야 한다!!!(물론 테이블을 나누고)

`-` 각 테이블에는 `기본키`가 있어야 하는데 `기본키는 일련번호`를 사용할 것이다

`-` `시도코드`와 `시도이름` 속성을 갖는 `Region` 테이블을 만들겠다(`시도코드 -> 기본키`)

`-` 결측치가 많은 컬럼은 `총콜레스테롤, 트리글리세라이드, HDL콜레스테롤, LDL콜레스테롤, 치아우식증유무, 치석, 음주여부`이다

`-` 컬럼의 속성을 콜레스테롤과 치아(Tooth)와 음주(Alcohol)로 나누어 테이블을 3개 만들겠다(음주는 애매하긴 한데 일단 독립적으로 만들겠다)

`-` `수축기혈압, 이완기혈압, 식전혈당(공복혈당), 혈청크레아티닌, (혈청지오티)AST, (혈청지오티)ALT`을 `Blood_Pressure_Sugar` 테이블로 만들겠다

`-` 나머지에 해당하는 `기준년도, 가입자일련번호, 성별코드, 연령대코드(5세단위), 시도코드, 신장(5Cm단위), 체중(5Kg단위), 허리둘레, 시력(좌), 시력(우), 청력(좌), 청력(우) 흡연상태`를 하나의 테이블로 만들겠다

`-` 여기서 `가입자일련번호 -> 기본키`, `시도코드 -> 외래키` 이다

- Region table

In [268]:
def get_region_table() -> pd.DataFrame:
    df = pd.DataFrame({'Region_Code':[0, 11, 42, 26, 43, 27, 44, 28, 45, 29, 46, 30, 47, 31, 48, 36, 49, 41],
                       'Region_Name':['전국', '서울특별시', '강원도', '부산광역시', '충청북도', '대구광역시', '충청남도',
                                      '인천광역시', '전라북도', '광주광역시', '전라남도', '대전광역시', '경상북도',
                                      '울산광역시', '경상남도', '세종특별자치시', '제주특별자치도', '경기도']})
    return df

In [269]:
region_table = get_region_table()

- Tooth table

`-` 치석 변수와 치아우식증유무 변수 중 하나라도 NA가 아닌 데이터로 table을 구성하겠다

In [173]:
def get_tooth_table(df: pd.DataFrame) -> pd.DataFrame:
    """치아우식증 유무 변수와 치석 변수 둘다 NA가 아닌 행만 추출하고 table 구성"""
    df_ = df.copy()
    df_ = df_.loc[(df_['치아우식증유무'].isnull()) & (df_['치석'].isnull()) == False, ['가입자일련번호', '치아우식증유무', '치석']] ## 가입자일련번호는 기본키
    df_ = df_.reset_index().drop('index', axis=1)
    return df_

In [160]:
tooth_table = get_tooth_table(df)

In [179]:
tooth_table.shape

(1129056, 3)

In [161]:
tooth_table.head()

,가입자일련번호,치아우식증유무,치석
0,2,0.0,0.0
1,3,0.0,0.0
2,4,0.0,0.0
3,6,0.0,1.0
4,11,0.0,1.0


- Alcohol table

In [192]:
def get_alcohol_table(df: pd.DataFrame) -> pd.DataFrame:
    """음주여부 변수가 NA가 아닌 행만 추출하고 table 구성"""
    df_ = df.copy()
    df_ = df_.loc[(df_['음주여부'].isnull()) == False, ['가입자일련번호', '음주여부']] ## 가입자일련번호는 기본키
    df_ = df_.reset_index().drop('index', axis=1)
    return df_

In [167]:
alcohol_table = get_alcohol_table(df)

In [178]:
alcohol_table.shape

(2289779, 2)

In [169]:
alcohol_table.head()

,가입자일련번호,음주여부
0,2,1.0
1,3,1.0
2,5,1.0
3,6,1.0
4,8,1.0


- Cholesterol table

In [200]:
def get_cholesterol_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    가입자일련번호, 총콜레스테롤, 트리글리세라이드, HDL콜레스테롤, LDL콜레스테롤 변수로 table 구성
    """
    columns = ['총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤'] ## 가입자일련번호는 기본키
    df_ = df.copy()
    df_ = df_.dropna(how='all', subset=columns) ## columns가 모두 NA인 행은 drop
    df_ = df_[['가입자일련번호'] + columns] ## 가입자일련번호는 기본키
    df_ = df_.reset_index().drop('index', axis=1)
    return df_

In [197]:
cholesterol_table = get_cholesterol_table(df)

In [198]:
cholesterol_table.shape

(1068659, 5)

In [199]:
cholesterol_table.head()

,가입자일련번호,총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤
0,4,234.0,90.0,80.0,135.0
1,7,171.0,168.0,34.0,103.0
2,8,186.0,72.0,60.0,111.0
3,9,189.0,60.0,87.0,90.0
4,11,134.0,64.0,53.0,68.0


- Blood Pressure and Sugar table

In [202]:
def get_blood_pressure_sugar_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    가입자일련번호, 수축기혈압, 이완기혈압, 식전혈당(공복혈당), 혈청크레아티닌, (혈청지오티)AST, (혈청지오티)ALT 변수로 table 구성
    """
    columns = ['수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT'] 
    df_ = df.copy()
    df_ = df_[['가입자일련번호'] + columns] ## 가입자일련번호는 기본키
    return df_

In [203]:
blood_pressure_sugar_table = get_blood_pressure_sugar_table(df)

In [205]:
blood_pressure_sugar_table.shape

(3000000, 7)

In [204]:
blood_pressure_sugar_table.head()

,가입자일련번호,수축기혈압,이완기혈압,식전혈당(공복혈당),혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT
0,1,110.0,60.0,96.0,0.5,21.0,8.0
1,2,128.0,78.0,79.0,1.0,35.0,45.0
2,3,128.0,65.0,80.0,1.1,77.0,47.0
3,4,151.0,89.0,100.0,0.6,19.0,10.0
4,5,114.0,62.0,124.0,0.5,26.0,26.0


- Basic Health table

In [210]:
def get_basic_health_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    *--- columns --**
    기준년도, 가입자일련번호, 성별코드, 연령대코드(5세단위),
    시도코드, 신장(5Cm단위), 체중(5Kg단위), 허리둘레,
    시력(좌), 시력(우), 청력(좌), 청력(우), 흡연상태
    
    위에 해당하는 변수들로 table 구성하고 반환
    """
    columns = ['기준년도', '성별코드', '연령대코드(5세단위)',
               '시도코드', '신장(5Cm단위)', '체중(5Kg단위)', '허리둘레',
               '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '흡연상태']
    
    df_ = df.copy()
    df_ = df_[['가입자일련번호'] + columns] ## 가입자일련번호는 기본키
    return df_

In [211]:
basic_health_table = get_basic_health_table(df)

In [212]:
basic_health_table.shape

(3000000, 13)

In [213]:
basic_health_table.head()

,가입자일련번호,기준년도,성별코드,연령대코드(5세단위),시도코드,신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),흡연상태
0,1,2018,2,7,48,160,60,79.5,1.5,1.5,1.0,1.0,1.0
1,2,2018,1,6,26,170,55,69.3,1.2,0.8,1.0,1.0,1.0
2,3,2018,1,12,28,165,70,85.0,0.8,0.8,2.0,1.0,2.0
3,4,2018,2,15,27,150,45,71.5,0.4,0.3,1.0,1.0,1.0
4,5,2018,2,14,41,145,50,77.0,0.7,0.6,1.0,1.0,1.0


## 행정구역별 성별 인구수 데이터 불러오기

In [290]:
def to_tidy_dataset(untidy_dataset: pd.DataFrame) -> pd.DataFrame:
    """
    KOSIS에서 제공하는 시계열 데이터셋과 같은 형식의 untidy 데이터를 tidy 데이터로 바꿔 반환
    데이터프레임의 형태가 동일하고 컬럼명만 다르다면 컬럼명만 수정하여 재사용 가능
    """ 
    tidy_dataset = untidy_dataset.set_index('행정구역(시군구)별').T.rename_axis(None, axis=1).\
                    reset_index().assign(Year = lambda df: df['index'].\
                    apply(lambda year: int(year[:year.find('.')]) if year.find('.') != -1 else int(year))).\
                    drop('index', axis=1).set_index(['Year', '행정구역(시군구)별']).\
                    applymap(lambda value: int(value)).\
                    stack().unstack(1).reset_index().rename_axis(None, axis=1).\
                    rename(columns={'level_1':'Region_Name', '남자인구수 (명)':'Male_Population', '여자인구수 (명)':'Female_Population', '총인구수 (명)':'Total_Population'})
    
    return tidy_dataset

In [304]:
def get_population_dataframe() -> pd.DataFrame:
    """
    행정구역_시군구별_성별_인구수.csv 파일을 불러온 후 tidy dataframe으로 변환한 후 전처리를 거치고 반환하는 함수
    
    *--- 전처리 ---*
    데이터프레임에 기본키이자 외래키인 시도코드 변수를 추가하고 시도이름 변수는 drop
    
    *--- Data Path ---*
    .(current directory)
    |--- Data
        |--- 행정구역_시군구별_성별_인구수.csv
    """
    df = pd.read_csv('./Data/행정구역_시군구별_성별_인구수.csv', encoding='euc-kr')
    df = to_tidy_dataset(df)
    df['Region_Code'] = df['Region_Name'].apply(lambda x: region_table.query(f'Region_Name == @x')['Region_Code'].item()) ## 시도코드 생성
    df = df.drop('Region_Name', axis=1) ## Region table에서 확인 가능하므로 drop
    df = df[['Year', 'Region_Code', 'Male_Population', 'Female_Population', 'Total_Population']] ## 기본키는 (연도, 시도코드), 외래키는 시도코드
    return df

In [302]:
df2 = get_population_dataframe()

In [303]:
df2.head()

,Year,Region_Code,Male_Population,Female_Population,Total_Population
0,2018,0,25866129,25959930,51826059
1,2018,11,4773899,4991724,9765623
2,2018,26,1692421,1749032,3441453
3,2018,27,1219342,1242427,2461769
4,2018,28,1481844,1472798,2954642


## 시군구별 의원 데이터 불러오기

`-` 이 데이터를 쓸지말지 고민이다

`-` 일단은 넘어가겠다

## DB 생성 및 데이터 업로드

- 변수이름 영문으로 변경

`-` 혹시 모를 error를 위해 테이블의 컬러명을 모두 영문으로 바꾸겠다

In [325]:
def rename_columns_to_english(df_list: pd.DataFrame) -> None:
    """컬럼명을 영문으로 바꿔주는 함수(원본이 변한다)"""
    name_dict = {'기준년도':'Year',
                 '가입자일련번호':'IDV_ID',
                 '성별코드':'Sex',
                 '연령대코드(5세단위)':'Age_Group',
                 '시도코드':'Region_Code',
                 '신장(5Cm단위)':'Height',
                 '체중(5Kg단위)':'Weight',
                 '허리둘레':'Waist',
                 '시력(좌)':'Sight_Left',
                 '시력(우)':'Sight_Right',
                 '청력(좌)':'Hear_Left',
                 '청력(우)':'Hear_Right',
                 '수축기혈압':'BP_High',
                 '이완기혈압':'BP_Low',
                 '식전혈당(공복혈당)':'BLDS',
                 '총콜레스테롤':'Total_Cholesterol',
                 '트리글리세라이드':'Triglyceride',
                 'HDL콜레스테롤':'HDL_Cholesterol',
                 'LDL콜레스테롤':'LDL_Cholesterol',
                 '혈색소':'Hemoglobin',
                 '요단백':'Proteinuria',
                 '혈청크레아티닌':'Serum_Creatinine',
                 '(혈청지오티)AST':'SGOT_AST',
                 '(혈청지오티)ALT':'SGOT_ALT',
                 '감마지티피':'Gamma_GTP',
                 '흡연상태':'Smoking_Type',
                 '음주여부':'Alcohol_YN',
                 '구강검진수검여부':'Dental_Inspec_YN',
                 '치아우식증유무':'CRS_YN',
                 '치석':'Tartar_YN'}
     
    for df in df_list:
        df.rename(columns=name_dict, inplace=True)

In [326]:
rename_columns_to_english([tooth_table,
                           alcohol_table,
                           cholesterol_table,
                           blood_pressure_sugar_table,
                           basic_health_table])

- DB 생성

In [400]:
## ./Data 폴더에 MedicalCheckup 데이터베이스를 생성하고 연결
conn = sqlite3.connect('./MedicalCheckup.db', isolation_level=None)

In [401]:
## 커서 생성
cur = conn.cursor()

- region table 생성

In [402]:
## Region 테이블 생성하기
sql = """CREATE TABLE IF NOT EXISTS Region  
        (
        Region_Code TEXT NOT NULL, 
        Region_Name TEXT NOT NULL,
        PRIMARY KEY (Region_Code)
        )
        """

cur.execute(sql)

In [403]:
# sql = 'DROP TABLE Region' ## table 삭제
# cur.execute(sql)

- basic health table 생성

In [409]:
## Basic_Health 테이블 생성하기
sql = """CREATE TABLE IF NOT EXISTS Basic_Health  
        (
        IDV_ID INTEGER NOT NULL, 
        Year INTEGER NOT NULL,
        Sex INTEGER NOT NULL,
        Age_Group INTEGER NOT NULL,
        Region_Code INTEGER NOT NULL,
        Height INTEGER NOT NULL,
        Weight INTEGER NOT NULL,
        Waist REAL,
        Sight_Left REAL,
        Sight_Right REAL,
        Hear_Left REAL,
        Hear_Right REAL,
        Smoking_Type INTEGER,
        PRIMARY KEY (IDV_ID),
        FOREIGN KEY (Region_Code) REFERENCES Region (Region_Code) 
        )
        """

cur.execute(sql)

In [408]:
# sql = 'DROP TABLE Basic_Health' ## table 삭제
# cur.execute(sql)

- tooth table 생성

In [406]:
## Tooth 테이블 생성하기
sql = """CREATE TABLE IF NOT EXISTS Tooth  
        (
        IDV_ID INTEGER NOT NULL, 
        CRS_YN INTEGER,
        Tartar_YN INTEGER,
        PRIMARY KEY (IDV_ID),
        FOREIGN KEY (IDV_ID) REFERENCES Basic_Health (IDV_ID) 
        )
        """

cur.execute(sql)

In [407]:
# sql = 'DROP TABLE Tooth' ## table 삭제
# cur.execute(sql)

- alcohol table 생성

In [417]:
## Alcohol 테이블 생성하기
sql = """CREATE TABLE IF NOT EXISTS Alcohol  
        (
        IDV_ID INTEGER NOT NULL, 
        Alcohol_YN INTEGER NOT NULL,
        PRIMARY KEY (IDV_ID),
        FOREIGN KEY (IDV_ID) REFERENCES Basic_Health (IDV_ID) 
        )
        """

cur.execute(sql)

In [418]:
# sql = 'DROP TABLE Alcohol' ## table 삭제
# cur.execute(sql)

- cholesterol table 생성

In [422]:
## Cholesterol 테이블 생성하기
sql = """CREATE TABLE IF NOT EXISTS Cholesterol 
        (
        IDV_ID INTEGER NOT NULL, 
        Total_Cholesterol REAL,
        Triglyceride REAL,
        HDL_Cholesterol REAL,
        LDL_Cholesterol REAL,
        PRIMARY KEY (IDV_ID),
        FOREIGN KEY (IDV_ID) REFERENCES Basic_Health (IDV_ID) 
        )
        """

cur.execute(sql)

In [423]:
# sql = 'DROP TABLE Cholesterol' ## table 삭제
# cur.execute(sql)

- blood pressure and sugar table 생성

In [424]:
## Blood_Pressure_Sugar 테이블 생성하기
sql = """CREATE TABLE IF NOT EXISTS Blood_Pressure_Sugar 
        (
        IDV_ID INTEGER NOT NULL, 
        BP_High INTEGER,
        BP_Low INTEGER,
        BLDS INTEGER,
        Serum_Creatinine REAL,
        SGOT_AST INTEGER,
        SGOT_ALT INTEGER,
        PRIMARY KEY (IDV_ID),
        FOREIGN KEY (IDV_ID) REFERENCES Basic_Health (IDV_ID) 
        )
        """

cur.execute(sql)

In [425]:
# sql = 'DROP TABLE Blood_Pressure_Sugar' ## table 삭제
# cur.execute(sql)

- population table 생성

In [428]:
## Population 테이블 생성하기
sql = """CREATE TABLE IF NOT EXISTS Population
        (
        Year INTEGER NOT NULL, 
        Region_Code INTEGER NOT NULL, 
        Male_Population INTEGER NOT NULL,
        Female_Population INTEGER NOT NULL,
        Total_Population INTEGER NOT NULL,
        PRIMARY KEY (Year, Region_Code)
        )
        """

cur.execute(sql)

In [429]:
# sql = 'DROP TABLE Population' ## table 삭제
# cur.execute(sql)

- 테이블 업로드